Proposal

A young chef, fresh out of Chef school wants to realize his dream and open a new restaurant in Toronto. He's facing two issues before opening his brand new restuarant:

1. Out of Toronto's 6 boroughs (Etobicoke, Scarborough, York, East York, North York, and the City of Toronto), where should he open the restaurant in order to maximize his profts?
2. As he's trained and proficient in a variety of different cuisines, what type of food should he serve once he chooses where to open his restaurant?

The ideal borough to build the restuarant will be one that has a high amount of people living in it since that'll mean that there would be more customers coming to the restuarant than an area with a low population.

We'll use data from https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/Table.cfm?Lang=Eng&T=1201&S=22&O=A to obtain population per FSA/bourough to determine which borough he should open his restaurant.

We'll also use the data below in combination with FourSquare API to cluster all the different restaurants in the boroughs to determine which is the most common type of food being served there. 

In [6]:
#Initilazing the data

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_321ef4e5eb254734813a631e9a5a9e25 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='K4qnAq3hg2a8s8QBLjdpHTUislPgN7EXGMFy8SWNwXih',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_321ef4e5eb254734813a631e9a5a9e25.get_object(Bucket='capstoneproject-donotdelete-pr-zqbqfzyhtcbpk7',Key='Toronto_FSA.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

Toronto_FSA = pd.read_csv(body)
Toronto_FSA.head(10)

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


In [7]:
#remove Bouroughs that have the value "Not assigned" and reindex
#In the wikipedia source, Neighborhoods have already been combined into 1 row
Clean_Toronto_FSA = Toronto_FSA.drop(Toronto_FSA[Toronto_FSA.Borough == 'Not assigned'].index).reset_index(drop=True)
Clean_Toronto_FSA.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [17]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [22]:
import requests # library to handle requests
CLIENT_ID = '1B5JTZDTUZIYBL0VTNWYMDQFFCJRGK2BZTRCVTQJDSNWYQD4'
CLIENT_SECRET = 'ETIBAJ4HAX2GD0BMR35QOEC0IHBKMODBFTY3XXBFHI2D5U0J' 
VERSION = '20200507'
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 

In [9]:
#Initialize population data
body = client_321ef4e5eb254734813a631e9a5a9e25.get_object(Bucket='capstoneproject-donotdelete-pr-zqbqfzyhtcbpk7',Key='FSA_Population.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

Population_Data = pd.read_excel(body)
Population_Data = Population_Data.rename(columns ={"PostalCode":'Postalcode'})
Population_Data.head()


,Postalcode,Population
0,A0A,46587.0
1,A0B,19792.0
2,A0C,12587.0
3,A0E,22294.0
4,A0G,35266.0


In [10]:
#Merge clean Toronto FSA data with population data
Toronto_Population = pd.merge(Toronto_FSA, Population_Data, how ='inner')
Toronto_Population.head()

,Postalcode,Borough,Neighborhood,Population
0,M3A,North York,Parkwoods,34615.0
1,M4A,North York,Victoria Village,14443.0
2,M5A,Downtown Toronto,Regent Park / Harbourfront,41078.0
3,M6A,North York,Lawrence Manor / Lawrence Heights,21048.0
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,10.0


In [11]:
#Now to sum up the population for each borough.
Toronto_Population_Sum = Toronto_Population.groupby('Borough').sum()
Toronto_Population_Sum.head(10)

,Population
Borough,
Central Toronto,164502.0
Downtown Toronto,271196.0
East Toronto,113966.0
East York,139996.0
Etobicoke,365318.0
North York,658944.0
Scarborough,634047.0
West Toronto,212782.0
York,171388.0


In [12]:
#Central Toronto, Downtown Toronto, East Toronto, and West Toronto are actually Toronto and should be combined into 1 Borough.
Toronto_Population2 = Toronto_Population
Toronto_Population2['Borough'] = Toronto_Population['Borough'].replace({'Central Toronto':'Toronto','Downtown Toronto':'Toronto','East Toronto':'Toronto','West Toronto':'Toronto'})
Toronto_Population2.head()
Toronto_Population_Sum2 = Toronto_Population2.groupby('Borough').sum()
Toronto_Population_Sum2.head(10)

,Population
Borough,
East York,139996.0
Etobicoke,365318.0
North York,658944.0
Scarborough,634047.0
Toronto,762446.0
York,171388.0


In [13]:
#now to sort the above data frame to make it easier to see which boroughs have a higher popultion
Toronto_Population_Sum2.sort_values(by=['Population'], ascending=False)

,Population
Borough,
Toronto,762446.0
North York,658944.0
Scarborough,634047.0
Etobicoke,365318.0
York,171388.0
East York,139996.0


Now let's see a map of the top 3 boroughs from above but first we need to append latitude and longitude to the data set

In [14]:

body = client_321ef4e5eb254734813a631e9a5a9e25.get_object(Bucket='capstoneproject-donotdelete-pr-zqbqfzyhtcbpk7',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
Geo_Code = pd.read_csv(body)
Geo_Code.head()
Geo_Code = Geo_Code.rename(columns={'Postal Code':'Postalcode'})
Geo_Code.head()
Toronto_Geocode = pd.merge(Toronto_Population2, Geo_Code, how = 'left')
Toronto_Geocode.head()

,Postalcode,Borough,Neighborhood,Population,Latitude,Longitude
0,M3A,North York,Parkwoods,34615.0,43.753259,-79.329656
1,M4A,North York,Victoria Village,14443.0,43.725882,-79.315572
2,M5A,Toronto,Regent Park / Harbourfront,41078.0,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,21048.0,43.718518,-79.464763
4,M7A,Toronto,Queen's Park / Ontario Provincial Government,10.0,43.662301,-79.389494


In [15]:
Toronto_Population2.head()

,Postalcode,Borough,Neighborhood,Population
0,M3A,North York,Parkwoods,34615.0
1,M4A,North York,Victoria Village,14443.0
2,M5A,Toronto,Regent Park / Harbourfront,41078.0
3,M6A,North York,Lawrence Manor / Lawrence Heights,21048.0
4,M7A,Toronto,Queen's Park / Ontario Provincial Government,10.0


In [16]:
#get the coordinates for the neighbourhoods in Toronto
Toronto_Neighbourhoods = Toronto_Geocode[Toronto_Geocode['Borough'] == 'Toronto'].reset_index(drop=True)
Toronto_Neighbourhoods.head()

,Postalcode,Borough,Neighborhood,Population,Latitude,Longitude
0,M5A,Toronto,Regent Park / Harbourfront,41078.0,43.654260,-79.360636
1,M7A,Toronto,Queen's Park / Ontario Provincial Government,10.0,43.662301,-79.389494
2,M5B,Toronto,Garden District / Ryerson,12785.0,43.657162,-79.378937
3,M5C,Toronto,St. James Town,2951.0,43.651494,-79.375418
4,M4E,Toronto,The Beaches,25044.0,43.676357,-79.293031


In [17]:
from geopy.geocoders import Nominatim

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of toronto are 43.6534817, -79.3839347.


In [21]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, label in zip(Toronto_Neighbourhoods['Latitude'], Toronto_Neighbourhoods['Longitude'], Toronto_Neighbourhoods['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [1]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [2]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
Toronto_venues = getNearbyVenues(names=Toronto_Neighbourhoods['Neighborhood'],latitudes=Toronto_Neighbourhoods['Latitude'],longitudes=Toronto_Neighbourhoods['Longitude'])
Toronto_venues.head()

Regent Park / Harbourfront
Queen's Park / Ontario Provincial Government
Garden District / Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
The Danforth West / Riverdale
Toronto Dominion Centre / Design Exchange
Brockton / Parkdale Village / Exhibition Place
India Bazaar / The Beaches West
Commerce Court / Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park / The Junction South
North Toronto West
The Annex / North Midtown / Yorkville
Parkdale / Roncesvalles
Davisville
University of Toronto / Harbord
Runnymede / Swansea
Moore Park / Summerhill East
Kensington Market / Chinatown / Grange Park
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Rosed

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park / Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park / Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,Regent Park / Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,Regent Park / Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [29]:
#Now I want to check how many venues we got for each neighbourhood in Toronto
Toronto_venues.groupby("Neighborhood").count().sort_values(by=['Venue'], ascending=False)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Toronto Dominion Centre / Design Exchange,100,100,100,100,100,100
Commerce Court / Victoria Hotel,100,100,100,100,100,100
First Canadian Place / Underground city,100,100,100,100,100,100
Garden District / Ryerson,100,100,100,100,100,100
Harbourfront East / Union Station / Toronto Islands,100,100,100,100,100,100
Stn A PO Boxes,94,94,94,94,94,94
Richmond / Adelaide / King,93,93,93,93,93,93
St. James Town,76,76,76,76,76,76
Church and Wellesley,73,73,73,73,73,73


In [36]:
#Now let's see how many unique categories of venues are in each neighbourhood in Toronto
len(Toronto_venues['Venue Category'].unique())

232

In [38]:
# one hot encoding
Toronto_venues2 = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_venues2['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_venues2.columns[-1]] + list(Toronto_venues2.columns[:-1])
Toronto_venues2 = Toronto_venues2[fixed_columns]

Toronto_venues2.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
Toronto_venues3 =  Toronto_venues2.groupby('Neighborhood').mean().reset_index()
Toronto_venues3.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.0,0.0
1,Brockton / Parkdale Village / Exhibition Place,0.043478,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,Business reply mail Processing Centre,0.055556,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.058824,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,Central Bay Street,0.016393,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.016393,0.0,0.0,0.0,0.0


In [47]:
#Time to find the top 10 venues in each neighbourhood
top_venues = 10

for neighbourhood in Toronto_venues3['Neighborhood']:
    print(neighbourhood)
    temp = Toronto_venues3[Toronto_venues3['Neighborhood'] == neighbourhood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Berczy Park
                 venue  freq
0          Coffee Shop  0.07
1         Cocktail Bar  0.05
2             Beer Bar  0.04
3                 Café  0.04
4   Seafood Restaurant  0.04
5          Cheese Shop  0.04
6               Bakery  0.04
7           Restaurant  0.04
8               Lounge  0.02
9  Sporting Goods Shop  0.02


Brockton / Parkdale Village / Exhibition Place
                    venue  freq
0                    Café  0.13
1          Breakfast Spot  0.09
2             Coffee Shop  0.09
3             Yoga Studio  0.04
4                  Bakery  0.04
5           Burrito Place  0.04
6            Climbing Gym  0.04
7       Convenience Store  0.04
8  Furniture / Home Store  0.04
9                     Gym  0.04


Business reply mail Processing Centre
                  venue  freq
0    Light Rail Station  0.11
1           Yoga Studio  0.06
2         Auto Workshop  0.06
3            Comic Shop  0.06
4           Pizza Place  0.06
5      Recording Studio  0.06
6            Resta

In [48]:
#Function to return the most common venue in each neighbourhood

def most_common(row, top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top_venues]

In [51]:
import numpy as np

top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Toronto_Neighbourhood_Top = pd.DataFrame(columns=columns)
Toronto_Neighbourhood_Top['Neighborhood'] = Toronto_venues3['Neighborhood']

for ind in np.arange(Toronto_venues3.shape[0]):
    Toronto_Neighbourhood_Top.iloc[ind, 1:] = most_common(Toronto_venues3.iloc[ind, :], top_venues)

Toronto_Neighbourhood_Top.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Restaurant,Café,Bakery,Beer Bar,Hotel,Eastern European Restaurant
1,Brockton / Parkdale Village / Exhibition Place,Café,Breakfast Spot,Coffee Shop,Grocery Store,Gym,Pet Store,Performing Arts Venue,Nightclub,Italian Restaurant,Intersection
2,Business reply mail Processing Centre,Light Rail Station,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Burrito Place,Recording Studio,Restaurant,Brewery
3,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Lounge,Coffee Shop,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boutique,Bar,Boat or Ferry
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Burger Joint,Thai Restaurant,Ice Cream Shop,Japanese Restaurant,Salad Place,Bubble Tea Shop
5,Christie,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Diner,Restaurant,Baby Store,Athletics & Sports,Nightclub
6,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Restaurant,Gastropub,Gay Bar,Hotel,Yoga Studio,Mediterranean Restaurant,Men's Store
7,Commerce Court / Victoria Hotel,Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Deli / Bodega,Japanese Restaurant,Italian Restaurant,Seafood Restaurant
8,Davisville,Sandwich Place,Café,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Gym,Sushi Restaurant,Park,Pharmacy
9,Davisville North,Department Store,Park,Gym,Breakfast Spot,Sandwich Place,Food & Drink Shop,Hotel,Women's Store,Eastern European Restaurant,Donut Shop


In [54]:
from sklearn.cluster import KMeans #import library that's need to perform cluster analysis

#Time to perform k means cluster analysis

#number of clusters
kclusters = 5

Toronto_Clusters = Toronto_venues3.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_Clusters)

In [57]:
Toronto_Neighbourhood_Top.insert(0, 'Cluster Labels', kmeans.labels_)

#staten_island_merged = Staten_Island_data

# merge staten_island_grouped with staten_island_data to add latitude/longitude for each neighborhood
#staten_island_merged = staten_island_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood').reset_index(drop=True)

Toronto_Neighbourhood_Top

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Restaurant,Café,Bakery,Beer Bar,Hotel,Eastern European Restaurant
1,0,Brockton / Parkdale Village / Exhibition Place,Café,Breakfast Spot,Coffee Shop,Grocery Store,Gym,Pet Store,Performing Arts Venue,Nightclub,Italian Restaurant,Intersection
2,0,Business reply mail Processing Centre,Light Rail Station,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Burrito Place,Recording Studio,Restaurant,Brewery
3,0,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Lounge,Coffee Shop,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boutique,Bar,Boat or Ferry
4,0,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Burger Joint,Thai Restaurant,Ice Cream Shop,Japanese Restaurant,Salad Place,Bubble Tea Shop
5,0,Christie,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Diner,Restaurant,Baby Store,Athletics & Sports,Nightclub
6,0,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Restaurant,Gastropub,Gay Bar,Hotel,Yoga Studio,Mediterranean Restaurant,Men's Store
7,0,Commerce Court / Victoria Hotel,Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Deli / Bodega,Japanese Restaurant,Italian Restaurant,Seafood Restaurant
8,0,Davisville,Sandwich Place,Café,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Gym,Sushi Restaurant,Park,Pharmacy
9,0,Davisville North,Department Store,Park,Gym,Breakfast Spot,Sandwich Place,Food & Drink Shop,Hotel,Women's Store,Eastern European Restaurant,Donut Shop
